In [ ]:
# Using bash, execute the following commands:

cd ${SPARK_HOME}

./sbin/start-master.sh

./sbin/start-slave.sh spark://SilverLining.localdomain:7077

# To stop master, run `./sbin/stop-master.sh`

In [1]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

pyspark.__version__

'3.0.3'

In [2]:
# Spark configuration
conf = SparkConf() \
    .set("spark.jars", "gcs-connector-hadoop3-2.2.5.jar") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", "~/.google/credentials/de-r-stocks.json")

In [3]:
# Spark context
sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", "~/.google/credentials/de-r-stocks.json")
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

22/04/08 16:33:15 WARN Utils: Your hostname, SilverLining resolves to a loopback address: 127.0.1.1; using 172.18.34.215 instead (on interface eth0)
22/04/08 16:33:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/04/08 16:33:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/08 16:33:19 WARN SparkContext: Please ensure that the number of slots available on your executors is limited by the number of cores to task cpus and not another custom resource. If cores is not the limiting resource then dynamic allocation will not work properly!


In [4]:
# Start Spark session
    # In production setting, master will be specified during spark-submit
spark = SparkSession.builder \
    .master('spark://SilverLining.localdomain:7077') \
    .appName('test') \
    .config(conf=sc.getConf()) \
    .getOrCreate()

In [ ]:
# Access data from GCS
df = spark.read.parquet('gs://datalake_de-r-stocks/stocks/submission/stocks_submission_2022-02-06.parquet')

To run the script on the cluster

```
spark-submit \
    --master="${URL}" \
    06_spark_sql.py \
        --input_green=data/pq/green/2021/*/ \
        --input_yellow=data/pq/yellow/2021/*/ \
        --output=data/report-2021
```